# Import library dan load dataset

pandas dan numpy digunakan untuk manipulasi dan analisis data, sedangkan dari sklearn, fungsi train_test_split digunakan untuk membagi dataset, StandardScaler untuk normalisasi fitur, dan accuracy_score untuk mengukur akurasi model. Selain itu, tensorflow diimpor untuk membangun model deep learning, dengan Sequential untuk arsitektur model dan layer Dense serta Dropout untuk mencegah overfitting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

DataFrame df diinisialisasi dengan memuat data dari file CSV bernama seattle-weather.csv menggunakan metode read_csv() dari library pandas

In [ ]:
df = pd.read_csv("seattle-weather.csv")

Metode head() digunakan untuk menampilkan lima baris pertama

In [ ]:
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


# Pre-processing

cek nilai null

In [ ]:
df.isna().sum()

,0
date,0
precipitation,0
temp_max,0
temp_min,0
wind,0
weather,0


menghapus kolom yang tidak dipakai untuk modeling

In [ ]:
# Menghapus kolom 'date'
df = df.drop(columns=['date'])

Encode
Targetnya adalah `weather`, kami ingin memprediksi apakah itu `rain`
Ubah `weather` menjadi klasifikasi biner: 1 jika `rain`, 0 jika tidak

In [ ]:
df['weather'] = df['weather'].apply(lambda x: 1 if x == 'rain' else 0)

# Model

## Normalize data and Split data

Dataset dipisahkan menjadi fitur (`X`) dan target (`y`), dengan `X` berisi semua kolom kecuali kolom `weather`, sementara `y` berisi data dari kolom `weather`. Fitur numerik pada `X` kemudian dinormalisasikan menggunakan `StandardScaler` agar skala datanya seragam. Setelah itu, dataset dibagi menjadi set pelatihan (`X_train`, `y_train`) dan set validasi (`X_val`, `y_val`) dengan proporsi 80% untuk pelatihan dan 20% untuk validasi, menggunakan metode `train_test_split` dengan `random_state` 42 untuk memastikan hasil yang konsisten.

In [ ]:
# Split features (X) and target (y)
X = df.drop(columns=['weather'])
y = df['weather']

# Normalisasikan fitur numerik
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Membagi dataset menjadi training set dan validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Build Model

Model neural network dibangun menggunakan **Sequential** dengan layer-layer sebagai berikut:
- layer input terhubung ke hidden layer pertama (64 neuron, aktivasi **ReLU**
- hidden layer kedua (32 neuron, aktivasi **ReLU**)
- layer **Dropout** (30% neuron dinonaktifkan untuk mencegah overfitting). Hidden layer ketiga memiliki 16 neuron, sedangkan output layer menggunakan 1 neuron dengan aktivasi **sigmoid** untuk binary classification.

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),# Input layer dan hidden layer ke-1
    Dense(32, activation='relu'), #hidden layer ke-2
    Dropout(0.3),  # Dropout layer (untuk mencegah overfitting), menonaktifkan 30% neuron secara acak
    Dense(16, activation='relu'),  # Hidden layer ketiga dengan 16 neuron, menggunakan aktivasi ReLU
    Dense(1, activation='sigmoid') # Output layer dengan 1 neuron, menggunakan aktivasi sigmoid (karena ini binary classification)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


summary arsitektur model

Metode summary() digunakan untuk menampilkan ringkasan struktur model neural network dan memberikan gambaran umum tentang arsitektur dan kompleksitas model yang dibangun.

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

## Train

Mengcompile model

Model dikompilasi menggunakan optimizer **Adam** dan fungsi loss yang digunakan adalah **binary_crossentropy**, sesuai untuk masalah klasifikasi biner. Selain itu, metrik **accuracy** ditambahkan untuk memantau akurasi model selama pelatihan dan evaluasi.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Membuat callback function untuk mencegah overfitt

Fitur **EarlyStopping** digunakan untuk menghentikan pelatihan model guna mencegah overfitting. Proses pelatihan akan dihentikan jika akurasi model tidak meningkat selama 3 epoch, yang ditentukan oleh parameter **patience**. Selain itu, parameter **restore_best_weights** diatur ke **True** untuk mengembalikan bobot model terbaik yang dicapai selama pelatihan.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Menghentikan pelatihan untuk mencegah overfit
early_stop = EarlyStopping(monitor='accuracy',
                           patience=3, # jika akurasi model tidak meningkat selama 3 epoch
                           restore_best_weights=True)

Menjalankan training model

Model dilatih menggunakan metode fit dengan data pelatihan X_train dan y_train selama 100 epoch dengan ukuran batch 32. Data validasi X_val dan y_val juga disertakan untuk memantau performa model selama pelatihan. Callback EarlyStopping ditambahkan untuk menghentikan pelatihan jika akurasi tidak meningkat setelah 3 epoch, membantu mencegah overfitting. Hasil pelatihan disimpan dalam variabel history

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=100, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9465 - loss: 0.1614 - val_accuracy: 0.9522 - val_loss: 0.1548
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9532 - loss: 0.1443 - val_accuracy: 0.9386 - val_loss: 0.1707
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9555 - loss: 0.1419 - val_accuracy: 0.9488 - val_loss: 0.1644
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9454 - loss: 0.1554 - val_accuracy: 0.9488 - val_loss: 0.1514
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9611 - loss: 0.1407 - val_accuracy: 0.9522 - val_loss: 0.1566
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9403 - loss: 0.1737 - val_accuracy: 0.9386 - val_loss: 0.1637
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9503 - loss: 0.1547 - val_accuracy: 0.9488 - val_loss: 0.1510
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9541 - loss: 0.1553 - val_accuracy: 0.9386 - v

## Evaluate

Model dievaluasi dengan menggunakan metode evaluate untuk menghitung kerugian dan akurasi pada data pelatihan (X_train, y_train) dan data validasi (X_val, y_val). Hasil evaluasi disimpan dalam variabel train_loss, train_acc untuk data pelatihan dan val_loss, val_acc untuk data validasi.

In [ ]:
# 6. Evaluasi Model
train_loss, train_acc = model.evaluate(X_train, y_train)
val_loss, val_acc = model.evaluate(X_val, y_val)

print(f"Training Accuracy: {train_acc*100:.2f}%")
print(f"Validation Accuracy: {val_acc*100:.2f}%")

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9586 - loss: 0.1327 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9432 - loss: 0.1728 
Training Accuracy: 96.06%
Validation Accuracy: 94.88%
